# Services

We've spun up pods and connected to them individually, but that's frankly not super useful if we want to distribute real traffic across those pods.

For instance, Pod is a execution unit and have a short life cycle:
- Kill if node is dead
- Recreated if rollout
- etc.


When Pod recreate, it will changing the IP, name, etc. which not good to setup a connection endpoint here and that why service come in

[Services](https://kubernetes.io/docs/concepts/services-networking/service/) provide a stable endpoint for pods. They are an abstraction used to provide a stable endpoint and load balance traffic across a group of Pods. By "stable endpoint", I just mean that the service will always be available at a given URL, even if the pod is destroyed and recreated.

It also introduce a concept called ***Load balancing***, primarily based on kube-proxy, a component running on each node in the cluster. Kube-proxy is responsible for monitoring changes to Services and Endpoints (a list of backend Pods), and then updating network rules to route traffic.

Create a file called web-service.yaml and add the following:
- `apiVersion: v1`
- `kind: Service`
- `metadata/name: web-service` (we could call it anything, but this is a fine name)
- `spec/selector/app`: I'm going to let you figure out what should be here. This is how the service knows which pods to route traffic to.
- spec/ports: An array of port objects. You need one entry:
    - `protocol: TCP` (TCP will allow us to use HTTP)
    - `port: 80` (this is the port that the service will listen on)
    - `targetPort: 8080` (this is the port that the pods are listening on)

This creates a new service called web-service with a few properties:
- It listens on port `80` for incoming traffic
- It forwards that traffic to pods that are listening on their port `8080`
- Its controller will continuously scan for pods matching the `app: synergychat-web` label selector and automatically add them to its pool

# Service Types

Take a look at the `yaml` we just created above:

```yaml
kubectl get svc web-service -o yaml
```

"svc" is a short-hand alias for "service", either will work in kubectl.

You should see a section that looks like this:

```yaml
spec:
  clusterIP: 10.96.213.234
  ...
  type: ClusterIP
```

Ok, we didn't specify it, but in short that is the default setting and only one type can be used at a time. Where the service type can be:
- **ClusterIP (default)**: Creating a virtual IP traffic from other Pods to the Service IP is distributed to backend Pods.
- **NodePort**: Opens ports on each node, similar to ClusterIP, but traffic from outside to the node port is also redirected and load-balanced.
- **LoadBalancer**: Integrates with the cloud provider's load balancer (such as AWS ELB, GCP Load Balancer). Kubernetes creates an external load balancer, and traffic is load-balanced by that provider before reaching the cluster. Inside the cluster, kube-proxy is still used for internal load balancing.
- **ExternalName**: Only DNS mapping. Return a CNAME record with that external hostname value. No proxying of any kind is set up.

The interesting thing about service types is that **they typically build on top of each other**.

For example, a `NodePort` service is just a `ClusterIP` service with the added functionality of exposing the service on each node's IP at a static port (it still has an internal cluster IP).

A `LoadBalancer` service is just a `NodePort` service with the added functionality of creating an external load balancer in the current cloud environment (it still has an internal cluster IP and node port).

An `ExternalName` service is actually a bit different. All it does is a DNS-level redirect. You can use it to redirect traffic from one service to another.

### ClusterIP Flow

```
[ Client Pod ]
      |
      |  DNS lookup
      v
web.default.svc.cluster.local
      |
      v
[ ClusterIP: 10.96.123.45 ]
      |
      |  kube-proxy (iptables / IPVS)
      v
[ Pod A: 10.244.1.5 ]
   or
[ Pod B: 10.244.2.7 ]
```

Where:
- ClusterIP:
    - is virtual IP
    - no listen port
- kube-proxy:
    - rewrite packet at kernel level
- Only used in cluster

### NodePort Flow

```
[ External Client ]
        |
        |  TCP connection
        v
[ NodeIP : NodePort ]
        |
        |  kube-proxy
        v
[ ClusterIP ]
        |
        |  kube-proxy (DNAT)
        v
[ Pod A / Pod B ]
```

- NodePort:
    - Open the same port on every Node
- Node don't have to run Pod
- Run above ClusterIP

### LoadBalancer Flow

```
[ Internet Client ]
        |
        |  DNS / IP
        v
[ Cloud Load Balancer ]
        |
        |  Forward traffic
        v
[ NodeIP : NodePort ]
        |
        |  kube-proxy
        v
[ ClusterIP ]
        |
        |  kube-proxy
        v
[ Pod A / Pod B ]
```

- Always go through a NodePort, take request from cloud provider

### ExternalName Flow

```
[ Client Pod ]
      |
      |  DNS lookup
      v
my-service.default.svc.cluster.local
      |
      v
CNAME → api.external.com
      |
      v
[ External Service ]
```

# API Service & Crawler Service

As you know, the `web` service in the SynergyChat system serves the front-end assets (HTML, CSS, and JavaScript) to the user's browser. The `api` service is responsible for handling requests from the front-end and returning data from the database.

Let's add a "NodePort" type service to expose the `api` service to the outside world.

Create a copy of your web-service.yaml file and name it `api-service.yaml`. Change the following:
- The name should be `api-service`
- Make sure it selects pods using the `app: synergychat-api` key
- Add `type: NodePort` to the `spec` section
- Add `nodePort: 30080` to the first object in the `ports` list

Here are the docs for the [NodePort](https://kubernetes.io/docs/concepts/services-networking/service/#type-nodeport) service type in case you get lost.

Once you've made the changes, apply the service. Then check to make sure the service is running:

```python
kubectl get svc
```

Then do the same with Crawler

# Change API Service

As you can see in 4 type of service above, `NodePort` and `LoadBalancer` services are mostly used to expose services to the outside world? That's true, but in most cloud-based Kubernetes environments, you'll actually use a Gateway object to expose your services. The Gateway object not only exposes your service to the outside world, but also allows you to do things like:
- Host multiple services on the same IP address
- Host multiple services on the same port (path-based routing)
- Terminate SSL
- Integrate directly with external DNS and load balancers

But in this tutorial, we don't have to do that, just to know when we would use different type to apply on real world application